# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [1]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

A continuación leemos los datasets en dataframes de Pandas.

In [2]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();

df_tsplib = pd.read_csv("experimentos/tsplib.csv");

La siguiente función sirve para correr el código sobre una instancia ejecutando un método en particular.
- FB: Fuerza bruta
- BT: Backtracking con ambas podas.
- BT-F: Backtracking solamente con poda por factibilidad.
- BT-O: Backtracking solamente con poda por optimalidad.
- DP: Programación dinámica.

In [3]:
def correr_experimento(*args):
    print(args)
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["../main", *args], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(F"Hubo un error en la experimentacion para el algoritmo: {args}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    [tamanio, costo_resultado] = process.stdout.read().split('\n')[0].split(" ");

    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return [tiempo_de_ejecucion, tamanio, costo_resultado];

## Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

In [10]:
experimentos = [];

## Experimento 1
Correr Heurisitica Golosa sobre todas las instancias de tsplib.

In [51]:
for index, row in df_tsplib.iterrows():
    experimentos.append([row["instancia"], row["ruta_instancia"], "H-CG"]);

## Experimento 2
Correr Heurisitica AGM sobre todas las instancias de tsplib.

In [52]:
for index, row in df_tsplib.iterrows():
    experimentos.append([row["instancia"], row["ruta_instancia"], "H-AGM"]);

## Experimento 3
Correr Busqueda Local sobre todas las instancias de tsplib.

In [53]:
for index, row in df_tsplib.iterrows():
    experimentos.append([row["instancia"], row["ruta_instancia"], "BL"]);

## Experimento 4
Correr Tabu Search sobre todas las instancias de tsplib.

In [11]:
for tipo_memoria in ['aristas', 'ciclos']:
    for T in range(3, 50, 3):
        for iteraciones in range(10000, 100000, 10000):
            for index, row in df_tsplib.iterrows():
                experimentos.append([row["instancia"], row["ruta_instancia"], "BL-Tabu", tipo_memoria, str(iteraciones), str(T)])

## Ejecutar los experimentos y guardar los resultados en un archivo CSV.
Este paso puede tardar unos minutos hasta terminar de ejecutarse.

In [12]:
columnas = ["instancia", "metodo", "tiempo", 'tamaño', 'costo'];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    tamanio_instancia = 0
    costo_resultado = 0
    T = 1 if experimento[2] == 'BL-Tabu' else 5
    for i in range(0, T):
        [tiempo, tamanio, costo] = correr_experimento(experimento[2], experimento[1], *experimento[3:])
        tiempos.append(tiempo);
        tamanio_instancia = tamanio
        costo_resultado = costo
    tiempo = np.median(tiempos);
    filas.append([experimento[0], experimento[2], tiempo, tamanio, costo, *experimento[3:]]);
    print([experimento[0], experimento[2], tiempo, tamanio, costo, *experimento[3:]])
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado.csv", index=False, header=True);

'Experimento: 13/19296'

('BL-Tabu', '../instancias/parseadas/d198.txt', 'aristas', '10000', '3')
('BL-Tabu', '../instancias/parseadas/d198.txt', 'aristas', '10000', '3')


KeyboardInterrupt: 